In [ ]:
import sympy
from control.matlab import *
from IPython.display import Latex, display
from sympy import Poly
from sympy.abc import s, z

In [1]:
%pylab %matplotlib inline

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


<div id="toc"></div>

# DC Motor Position: Digital Controller Design

In this example, the controller will be designed applying a Root Locus method. A sample-data DC motor model can be obtained from conversion of an analog DC motor model, as we will describe. According to the [DC Motor Position: System Modeling](../MotorPosition/MotorPosition-SystemModeling.ipynb) page, the continuos open-loop transfer function for DC motor's position in the Laplace domain is the following.

$$
P(s) = \frac {\Theta(s)}{V(s)} =  \frac{K}{s ( (Js + b)(Ls + R) + K^2 )} \qquad  [ \frac{rad}{V} ]
$$

The structure of the control system has the form shown in the figure below.
![feedback_motorp.png](figures/feedback_motorp.png)
Also from the main page, the design requirements for a 1-radian step reference are the following.
* Settling time less than 0.040 seconds
* Overshoot less than 16
* No steady-state error, even in the presence of a step disturbance input

## Creating a sampled-date model of the plant

The first step in the design of a digital control system is to generate a sampled-data model of the plant. Therefore, it is necessary to choose a frequency with which the continuous-time plant is sampled. In choosing a sampling period, it is desired that the sampling frequency be fast compared to the dynamics of the system in order that the sampled output of the system captures the system's full behavior, that is, so that significant inter-sample behavior isn't missed. Let's create a continuous-time model of the plant. Create a new [m-file](../Extras_Mfile.ipynb) and add the following MATLAB code (refer to the main problem for the details of getting these commands). Running the m-file within the MATLAB command window will generate the output shown below.

In [2]:
# J = 3.2284E-6;
# b = 3.5077E-6;
# K = 0.0274;
# R = 4;
# L = 2.75E-6;
# s = tf('s');
# P_motor = K/(s*((J*s+b)*(L*s+R)+K^2));
# zpk(P_motor)

The use of the |zpk| command above transforms the transfer function into a form where the zeros, poles, and gain can be seen explicitly. Examining the poles of the plant (or its frequency response), it is clear that the pole at -1.45e06 contributes very little to the response of the plant. The gain crossover frequency of the plant is approximately 5 Hz. Therefore, choosing a sampling period of 0.001 seconds (frequency of 1000 Hz) is significantly faster than the dynamics of the plant. This sampling period is also fast compared to the speed that will be achieved by the resultant closed-loop system. A sample time of 0.001 seconds is specifically 1/100 of the required time constant and 1/40 of the required settling time. In this case, we will convert the given transfer function from the continuous Laplace domain to the discrete z-domain. MATLAB can be used to achieve this conversion through the use of the |c2d| command. The |c2d| command requires three arguments: a system model, the sampling time (|Ts|), and the type of hold circuit. In this example, we will assume a zero-order hold (|zoh|) circuit. Refer to the [Introduction: Digital Controller Design](../Introduction/Introduction-ControlDigital.ipynb) page for further details. Adding the following commands to your m-file and running in the MATLAB command window generates the sampled-data model shown below.

In [3]:
# Ts = 0.001;
# dP_motor = c2d(P_motor, Ts, 'zoh');
# zpk(dP_motor)

From the above, notice that there is a pole and zero very near to _z_ = 0 that effectively cancel. This cancellation in the transfer function can be accomplished by applying the |minreal| command with a tolerance of 0.001. Cancellation of this pole and zero will reduce the order of our transfer function and will help to avoid numerical difficulties in MATLAB. Applying the |minreal| command, therefore, produces the following reduced order transfer function. Note the absence of the pole and zero near _z_ = 0.

In [4]:
# dP_motor = minreal(dP_motor,0.001);
# zpk(dP_motor)

We would now like to analyze the closed-loop response of the system without any additional compensation. First, we have to close the loop of the transfer function by using the |feedback| command. After closing the loop, let's inspect the closed-loop step response with the zero-order hold. This can be accomplished by using the |step| and |stairs| commands. Since the |step| command is fed a discrete model, it will output a vector of discrete samples at the sample time |Ts| associated with the model (click [here](../Extras_step.ipynb) for further details). The syntax below specifies to simulate the step response for 0.5 seconds. The |stairs| command draws these discrete data points as a stairstep, just like what would be produced by a zero-order hold circuit. Add the following code at the end of your previous m-file and rerun it. You should generate a plot like the one shown below.

In [5]:
# sys_cl = feedback(dP_motor,1);
# [x1,t] = step(sys_cl,.5);
# stairs(t,x1)
# xlabel('Time (seconds)')
# ylabel('Position (radians)')
# title('Stairstep Response: Original')
# grid

In [6]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>